In [1]:
import pandas as pd
import numpy as np
import json
from sklearn import preprocessing

In [2]:
input_path = '../data/data.csv'
output_path = '../data/grouped_data.csv'
# input_path = '../data/data1.csv'
# output_path = '../data/grouped_data1.csv'
# input_path = '../data/data2.csv'
# output_path = '../data/grouped_data2.csv'

In [3]:
df = pd.read_csv(input_path)

In [4]:
# get the difference of each features 
df['diff_dwelling_time_1'] = df['dwelling_time_2'] - df['dwelling_time_1']
df['diff_dwelling_time_2'] = df['dwelling_time_3'] - df['dwelling_time_2']
df['diff_sd_current_1'] = df['sd_current_2'] - df['sd_current_1']
df['diff_sd_current_2'] = df['sd_current_3'] - df['sd_current_2']
df['diff_mean_current_1'] = df['mean_current_2'] - df['mean_current_1']
df['diff_mean_current_2'] = df['mean_current_3'] - df['mean_current_2']

In [5]:
# aggregate and find min, max, median, std for each features

lst = ['transcript_id', 'transcript_position', 'sevenmers']

grouped_df = df.groupby(by = lst).agg(
    {'dwelling_time_1': [min, max, 'median', 'std'],
     'sd_current_1': [min, max, 'median', 'std'],
     'mean_current_1': [min, max, 'median', 'std'],
     'dwelling_time_2': [min, max, 'median', 'std'],
     'sd_current_2': [min, max, 'median', 'std'],
     'mean_current_2': [min, max, 'median', 'std'],
     'dwelling_time_3': [min, max, 'median', 'std'],
     'sd_current_3': [min, max, 'median', 'std'],
     'mean_current_3': [min, max, 'median', 'std'],
     'diff_dwelling_time_1': [min, max, 'median', 'std'],
     'diff_dwelling_time_2': [min, max, 'median', 'std'],
     'diff_sd_current_1': [min, max, 'median', 'std'],
     'diff_sd_current_2': [min, max, 'median', 'std'],
     'diff_mean_current_1': [min, max, 'median', 'std'],
     'diff_mean_current_2': [min, max, 'median', 'std'],
    }).reset_index()

In [6]:
# rename the columns
grouped_df2 = grouped_df
grouped_df2.columns = ["_".join(x) for x in np.ravel(grouped_df.columns)]
grouped_df2 = grouped_df.rename(columns = {'transcript_id_': 'transcript_id', 
                                           'transcript_position_': 'transcript_position',
                                           'sevenmers_': 'sevenmers'})

In [7]:
grouped_df2

,transcript_id,transcript_position,sevenmers,dwelling_time_1_min,dwelling_time_1_max,dwelling_time_1_median,dwelling_time_1_std,sd_current_1_min,sd_current_1_max,sd_current_1_median,...,diff_sd_current_2_median,diff_sd_current_2_std,diff_mean_current_1_min,diff_mean_current_1_max,diff_mean_current_1_median,diff_mean_current_1_std,diff_mean_current_2_min,diff_mean_current_2_max,diff_mean_current_2_median,diff_mean_current_2_std
0,ENST00000000233,244,AAGACCA,0.00199,0.0339,0.00697,0.005399,1.770,13.40,3.73,...,-2.690,4.595526,-16.0,24.0,1.0,5.411732,-54.9,-31.8,-45.40,3.559291
1,ENST00000000233,261,CAAACTG,0.00199,0.0222,0.00564,0.003599,0.919,17.00,2.88,...,-0.315,2.003694,-13.9,11.1,-2.0,4.507050,-21.6,-2.3,-13.60,3.777883
2,ENST00000000233,316,GAAACAG,0.00232,0.0299,0.00631,0.004456,1.280,11.60,2.65,...,-1.740,1.157738,-16.0,2.0,-6.0,3.041505,-17.6,1.7,-10.00,3.194735
3,ENST00000000233,332,AGAACAT,0.00232,0.0370,0.00902,0.006136,1.220,14.20,5.73,...,-0.505,1.498928,-39.7,-14.8,-31.8,3.443895,-24.3,-1.1,-7.50,4.004789
4,ENST00000000233,368,AGGACAA,0.00199,0.0478,0.00896,0.007169,1.150,39.00,6.52,...,-1.365,2.317262,-10.0,11.0,4.0,2.993918,-53.4,-28.3,-36.65,3.159567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,GGGACAT,0.00232,0.0432,0.00817,0.006178,0.924,6.00,3.20,...,-0.860,3.086679,-18.0,10.0,-2.0,4.771331,-46.0,-13.2,-34.80,5.263465
121834,ENST00000641834,1429,CTGACAC,0.00232,0.0262,0.00618,0.005625,1.770,10.30,3.69,...,-5.400,2.252558,-13.0,16.0,4.0,5.489145,-46.0,-18.4,-34.70,3.998798
121835,ENST00000641834,1531,TGGACAC,0.00232,0.0315,0.00697,0.005521,1.400,6.77,3.83,...,-2.650,2.254478,-10.0,11.0,0.0,3.693490,-35.9,-20.0,-29.20,3.088610
121836,ENST00000641834,1537,CTGACCA,0.00232,0.0483,0.00660,0.007090,1.460,8.48,3.16,...,-3.740,2.468316,3.0,29.0,13.0,5.409261,-54.7,-32.7,-41.10,3.906063


In [8]:
# find the relative position of each read in each transcript
grouped_df2['relative_position'] = grouped_df2.groupby('transcript_id')['transcript_position'].transform(lambda x: (x - x.min())/(x.max()-x.min()))

In [9]:
# note: have NAs because there's transcripts with only one position
# fill the NAs with 0
grouped_df2['relative_position'] = grouped_df2['relative_position'].fillna(0)

In [10]:
# split the sevenmers into seven columns
order_df = pd.DataFrame(grouped_df2['sevenmers'].str.split('').to_list())[[x for x in range(1, 8)]].rename(
    columns = {1: 'order_1', 2:'order_2', 
               3: 'order_3', 4:'order_4', 
               5: 'order_5', 6:'order_6',
               7: 'order_7'})
grouped_df3 = pd.concat([grouped_df2, order_df], axis = 1)
grouped_df3

,transcript_id,transcript_position,sevenmers,dwelling_time_1_min,dwelling_time_1_max,dwelling_time_1_median,dwelling_time_1_std,sd_current_1_min,sd_current_1_max,sd_current_1_median,...,diff_mean_current_2_median,diff_mean_current_2_std,relative_position,order_1,order_2,order_3,order_4,order_5,order_6,order_7
0,ENST00000000233,244,AAGACCA,0.00199,0.0339,0.00697,0.005399,1.770,13.40,3.73,...,-45.40,3.559291,0.000000,A,A,G,A,C,C,A
1,ENST00000000233,261,CAAACTG,0.00199,0.0222,0.00564,0.003599,0.919,17.00,2.88,...,-13.60,3.777883,0.025411,C,A,A,A,C,T,G
2,ENST00000000233,316,GAAACAG,0.00232,0.0299,0.00631,0.004456,1.280,11.60,2.65,...,-10.00,3.194735,0.107623,G,A,A,A,C,A,G
3,ENST00000000233,332,AGAACAT,0.00232,0.0370,0.00902,0.006136,1.220,14.20,5.73,...,-7.50,4.004789,0.131540,A,G,A,A,C,A,T
4,ENST00000000233,368,AGGACAA,0.00199,0.0478,0.00896,0.007169,1.150,39.00,6.52,...,-36.65,3.159567,0.185351,A,G,G,A,C,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,GGGACAT,0.00232,0.0432,0.00817,0.006178,0.924,6.00,3.20,...,-34.80,5.263465,0.737842,G,G,G,A,C,A,T
121834,ENST00000641834,1429,CTGACAC,0.00232,0.0262,0.00618,0.005625,1.770,10.30,3.69,...,-34.70,3.998798,0.799392,C,T,G,A,C,A,C
121835,ENST00000641834,1531,TGGACAC,0.00232,0.0315,0.00697,0.005521,1.400,6.77,3.83,...,-29.20,3.088610,0.876900,T,G,G,A,C,A,C
121836,ENST00000641834,1537,CTGACCA,0.00232,0.0483,0.00660,0.007090,1.460,8.48,3.16,...,-41.10,3.906063,0.881459,C,T,G,A,C,C,A


In [11]:
columns = ['order_1', 'order_2', 'order_3', 'order_4', 'order_5', 'order_6', 'order_7']
for col in columns:
    print(f'{col}: {grouped_df3[col].unique()}')

order_1: ['A' 'C' 'G' 'T']
order_2: ['A' 'G' 'T']
order_3: ['G' 'A']
order_4: ['A']
order_5: ['C']
order_6: ['C' 'T' 'A']
order_7: ['A' 'G' 'T' 'C']


In [12]:
# order_4 is always A, order_5 is always C --> drop
grouped_df3 = grouped_df3.drop(columns=['order_4', 'order_5'])

In [13]:
# find the number of occurrence of a letter in a word
# input: str word, str letter
# output: int
def find(word, letter):
    res = 0
    for i in word:
        if i==letter:
            res += 1
    return res

In [14]:
# count the A,C,G,T in the sevenmers
grouped_df3['count_A'] = grouped_df3['sevenmers'].map(lambda x: find(x, 'A'))
grouped_df3['count_C'] = grouped_df3['sevenmers'].map(lambda x: find(x, 'C'))
grouped_df3['count_G'] = grouped_df3['sevenmers'].map(lambda x: find(x, 'G'))
grouped_df3['count_T'] = grouped_df3['sevenmers'].map(lambda x: find(x, 'T'))

In [15]:
grouped_df3

,transcript_id,transcript_position,sevenmers,dwelling_time_1_min,dwelling_time_1_max,dwelling_time_1_median,dwelling_time_1_std,sd_current_1_min,sd_current_1_max,sd_current_1_median,...,relative_position,order_1,order_2,order_3,order_6,order_7,count_A,count_C,count_G,count_T
0,ENST00000000233,244,AAGACCA,0.00199,0.0339,0.00697,0.005399,1.770,13.40,3.73,...,0.000000,A,A,G,C,A,4,2,1,0
1,ENST00000000233,261,CAAACTG,0.00199,0.0222,0.00564,0.003599,0.919,17.00,2.88,...,0.025411,C,A,A,T,G,3,2,1,1
2,ENST00000000233,316,GAAACAG,0.00232,0.0299,0.00631,0.004456,1.280,11.60,2.65,...,0.107623,G,A,A,A,G,4,1,2,0
3,ENST00000000233,332,AGAACAT,0.00232,0.0370,0.00902,0.006136,1.220,14.20,5.73,...,0.131540,A,G,A,A,T,4,1,1,1
4,ENST00000000233,368,AGGACAA,0.00199,0.0478,0.00896,0.007169,1.150,39.00,6.52,...,0.185351,A,G,G,A,A,4,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,GGGACAT,0.00232,0.0432,0.00817,0.006178,0.924,6.00,3.20,...,0.737842,G,G,G,A,T,2,1,3,1
121834,ENST00000641834,1429,CTGACAC,0.00232,0.0262,0.00618,0.005625,1.770,10.30,3.69,...,0.799392,C,T,G,A,C,2,3,1,1
121835,ENST00000641834,1531,TGGACAC,0.00232,0.0315,0.00697,0.005521,1.400,6.77,3.83,...,0.876900,T,G,G,A,C,2,2,2,1
121836,ENST00000641834,1537,CTGACCA,0.00232,0.0483,0.00660,0.007090,1.460,8.48,3.16,...,0.881459,C,T,G,C,A,2,3,1,1


# Join with Labels

In [16]:
# only data.csv has labels and gene_id
if (input_path == '../data/data.csv'):
    data = pd.read_csv('../data/data.info')
    grouped_df3 = grouped_df3.merge(data, how='left', on=['transcript_id', 'transcript_position'])
    gene_id_col = grouped_df3.pop('gene_id')
    grouped_df3.insert(0, 'gene_id', gene_id_col)

In [17]:
grouped_df3

,gene_id,transcript_id,transcript_position,sevenmers,dwelling_time_1_min,dwelling_time_1_max,dwelling_time_1_median,dwelling_time_1_std,sd_current_1_min,sd_current_1_max,...,order_1,order_2,order_3,order_6,order_7,count_A,count_C,count_G,count_T,label
0,ENSG00000004059,ENST00000000233,244,AAGACCA,0.00199,0.0339,0.00697,0.005399,1.770,13.40,...,A,A,G,C,A,4,2,1,0,0
1,ENSG00000004059,ENST00000000233,261,CAAACTG,0.00199,0.0222,0.00564,0.003599,0.919,17.00,...,C,A,A,T,G,3,2,1,1,0
2,ENSG00000004059,ENST00000000233,316,GAAACAG,0.00232,0.0299,0.00631,0.004456,1.280,11.60,...,G,A,A,A,G,4,1,2,0,0
3,ENSG00000004059,ENST00000000233,332,AGAACAT,0.00232,0.0370,0.00902,0.006136,1.220,14.20,...,A,G,A,A,T,4,1,1,1,0
4,ENSG00000004059,ENST00000000233,368,AGGACAA,0.00199,0.0478,0.00896,0.007169,1.150,39.00,...,A,G,G,A,A,4,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENSG00000167747,ENST00000641834,1348,GGGACAT,0.00232,0.0432,0.00817,0.006178,0.924,6.00,...,G,G,G,A,T,2,1,3,1,1
121834,ENSG00000167747,ENST00000641834,1429,CTGACAC,0.00232,0.0262,0.00618,0.005625,1.770,10.30,...,C,T,G,A,C,2,3,1,1,0
121835,ENSG00000167747,ENST00000641834,1531,TGGACAC,0.00232,0.0315,0.00697,0.005521,1.400,6.77,...,T,G,G,A,C,2,2,2,1,1
121836,ENSG00000167747,ENST00000641834,1537,CTGACCA,0.00232,0.0483,0.00660,0.007090,1.460,8.48,...,C,T,G,C,A,2,3,1,1,0


# Normalisation

In [18]:
grouped_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121838 entries, 0 to 121837
Data columns (total 75 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   gene_id                      121838 non-null  object 
 1   transcript_id                121838 non-null  object 
 2   transcript_position          121838 non-null  int64  
 3   sevenmers                    121838 non-null  object 
 4   dwelling_time_1_min          121838 non-null  float64
 5   dwelling_time_1_max          121838 non-null  float64
 6   dwelling_time_1_median       121838 non-null  float64
 7   dwelling_time_1_std          121838 non-null  float64
 8   sd_current_1_min             121838 non-null  float64
 9   sd_current_1_max             121838 non-null  float64
 10  sd_current_1_median          121838 non-null  float64
 11  sd_current_1_std             121838 non-null  float64
 12  mean_current_1_min           121838 non-null  float64
 13 

In [19]:
numerical_columns = grouped_df3.select_dtypes(include=['int64', 'float64']).columns
string_columns = grouped_df3.select_dtypes(include=['object']).columns

In [20]:
# normalise the numerical columns
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(grouped_df3[numerical_columns])
df_normalised = pd.DataFrame(x_scaled)
df_normalised.columns = numerical_columns

In [21]:
final_df = pd.concat([grouped_df3[string_columns], df_normalised], axis=1)
final_df

,gene_id,transcript_id,sevenmers,order_1,order_2,order_3,order_6,order_7,transcript_position,dwelling_time_1_min,...,diff_mean_current_2_min,diff_mean_current_2_max,diff_mean_current_2_median,diff_mean_current_2_std,relative_position,count_A,count_C,count_G,count_T,label
0,ENSG00000004059,ENST00000000233,AAGACCA,A,A,G,C,A,0.012869,0.040097,...,0.300138,0.171181,0.186594,0.134701,0.000000,0.6,0.333333,0.25,0.00,0.0
1,ENSG00000004059,ENST00000000233,CAAACTG,C,A,A,T,G,0.013777,0.040097,...,0.760719,0.490791,0.762681,0.145936,0.025411,0.4,0.333333,0.25,0.25,0.0
2,ENSG00000004059,ENST00000000233,GAAACAG,G,A,A,A,G,0.016714,0.080194,...,0.816044,0.534128,0.827899,0.115964,0.107623,0.6,0.000000,0.50,0.00,0.0
3,ENSG00000004059,ENST00000000233,AGAACAT,A,G,A,A,T,0.017568,0.080194,...,0.723375,0.503792,0.873188,0.157599,0.131540,0.6,0.000000,0.25,0.25,0.0
4,ENSG00000004059,ENST00000000233,AGGACAA,A,G,G,A,A,0.019491,0.040097,...,0.320885,0.209101,0.345109,0.114156,0.185351,0.6,0.000000,0.50,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENSG00000167747,ENST00000641834,GGGACAT,G,G,G,A,T,0.071821,0.080194,...,0.423237,0.372698,0.378623,0.222292,0.737842,0.2,0.000000,0.75,0.25,1.0
121834,ENSG00000167747,ENST00000641834,CTGACAC,C,T,G,A,C,0.076147,0.080194,...,0.423237,0.316360,0.380435,0.157291,0.799392,0.2,0.666667,0.25,0.25,0.0
121835,ENSG00000167747,ENST00000641834,TGGACAC,T,G,G,A,C,0.081593,0.080194,...,0.562932,0.299025,0.480072,0.110509,0.876900,0.2,0.333333,0.50,0.25,1.0
121836,ENSG00000167747,ENST00000641834,CTGACCA,C,T,G,C,A,0.081914,0.080194,...,0.302905,0.161430,0.264493,0.152524,0.881459,0.2,0.666667,0.25,0.25,0.0


In [22]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121838 entries, 0 to 121837
Data columns (total 75 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   gene_id                      121838 non-null  object 
 1   transcript_id                121838 non-null  object 
 2   sevenmers                    121838 non-null  object 
 3   order_1                      121838 non-null  object 
 4   order_2                      121838 non-null  object 
 5   order_3                      121838 non-null  object 
 6   order_6                      121838 non-null  object 
 7   order_7                      121838 non-null  object 
 8   transcript_position          121838 non-null  float64
 9   dwelling_time_1_min          121838 non-null  float64
 10  dwelling_time_1_max          121838 non-null  float64
 11  dwelling_time_1_median       121838 non-null  float64
 12  dwelling_time_1_std          121838 non-null  float64
 13 

In [23]:
# print column name if there's null values in it
for name in final_df.columns:
    if (final_df[name].isnull().any()):
        print(name)

# Export to csv

In [24]:
final_df.to_csv(output_path, index=False)